# APMTH 207: Advanced Scientific Computing: 
## Stochastic Methods for Data Analysis, Inference and Optimization
## Homework #9
**Harvard University**<br>
**Spring 2017**<br>
**Instructors: Rahul Dave**<br>
**Due Date: ** Friday, April 7th, 2017 at 11:59pm

**Instructions:**

- Upload your final answers as well as your iPython notebook containing all work to Canvas.

- Structure your notebook and your work to maximize readability.

## Question 1. Separate the Bayesian Irises

The goal of this problem is to introduce you to the idea of classification 
using Bayesian inferences. 

You are given the famous *Fisher flower Iris data set*
which is a  multivariate data set introduced by Sir Ronald Fisher (1936) as an example of discriminant analysis.
The data set consists of 50 samples from each of three species of Iris (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimeters. Based on the combination of these four features, you will build a model to predict
the species. 

For this problem only consider two classes: **virginica** and **not-virginica**. 

The iris data can be obtained [here](iris.txt).

Lets $(X, Y )$ be our dataset, where $X=\{\vec{x}_1, \ldots \vec{x}_n\}$ and $\vec{x}_i$ is a 5D vector corresponding to an offset 1 and
the four components explained above. $Y \in \{0,1\}$ are the scalar 
labels of a class. In other words  
the species labels are your $Y$ data (virginica = 0 and virginica=1), and the four features, petal length
and width, sepal length and width, along with the offset, are your $X$ data. 

The goal is to train a classifier, that will predict an unknown class label $\hat{y}$ from a new data point $x$. 

Consider the following glm (logistic model) for the probability of a class:

$$ p(y) = \frac{1}{1+e^{-x^T \beta}} $$

(or $logit(p) = x^T \beta$ in more traditional glm form)

where $\beta$ is a 5D parameter to learn. 

Then given $p$ at a particular data point $x$, we can use a bernoulli likelihood to get 1's and 0's. This should be enough for you to set up your model in pymc3. (Other Hints: also use theano.tensor.exp when you define the inerse logit to go from $\beta$ to $p$, and you might want to set up $p$ as a deterministic explicitly so that pymc3 does the work of giving you the trace).

Use a 60-40 stratified (preserving class membership) split of the dataset into a training set and a test set. ( See the docs for scikit-learn's `train_test_split`).

## Part A: Sampling

1. Choose a prior for $\beta \sim N(0, \sigma^2 I) $ and write down the formula for the normalized posterior $p(\beta| Y,X)$. Since we dont care about regularization here, just use the mostly uninformative value $\sigma = 10$.
2. Find the MAP and mean estimate for the posterior on the training set.
3. Implement a  sampler to sample from this posterior of $\beta$.   Generate samples of $\beta$ and plot the sequence of $\beta$'s  and histograms for each $\beta$ component.
4. Use both the MAP and MEAN estimate of $\beta$, and the posterior distribution of $\beta$, to generate a set of probabilities and a set of posterior distributions respectively, for each data point $x$ in the training set. Plot the posterior against the MAP and MEAN estimate for 2 or 3 of your data points, preferably those with MAP estimates closer to 0.5.

---

Now that we have both point estimate(s) and a posterior distribution on the probability of a data point being virginica, we can use these to make predictions on both the training set and test set.

**There are two ways to make these predictions, given an estimate of $p(y=1 \vert x)$ **: 

(A) You can sample from the Bernoulli likelihood at the data point $x$ to decide if that particular data points classification $y(x)$ should be a 1 or a 0.

(B) Or you could do the intuitive "machine-learning-decision-theoretic" (MLDT) thing, where you assign a data  point $x$ a classification 1 if $p(y=1 \vert x) > 0.5$.

**And there are multiple ways in which you can do these probability estimates at a sample $x$**:

(A) You could just use the MAP value, ($p_{MAP}$), or the posterior mean ($p_{MEAN}$)

(B) You can see what fraction of your posterior samples have values above 0.5 (ie you are calculating 1-cdf(0.5)) on the posterior ($p_{CDF}$))

(C) Both these above methods miss the combined smearing of the posterior and sampling distributions. In other words they dont sample from the posterior predictive. If we draw a large number of samples from the posterior predictive distribution at a data point $x$, the fraction of 1s will give an estimate for the probability to use, $p_{PP}$, which is different from the MAP estimate, or the CDF estimate.

## Part B: Playing with Probabilities

1. Plot the distribution of $p_{MEAN}$, $p_{CDF}$, and $p_{PP}$ over all the data points in the training set. How are these different?
2. Plot the posterior-predictive distribution of the misclassification rate with respect to the true class identities $y(x)$ of the data points $x$ (in other words you are plotting a histogram with the misclassification rate for the $n_{trace}$ posterior-predictive samples) on the training set.
3. Make the same plot from the posterior, rather than the posterior predictive, by using the MLDT defined above. Overlay this plot on the previous one. That is, for every posterior sample, consider whether the data point ought to be classified as a 1 or 0 from the $p>0.5 \implies y=1$ decision theoretic prespective. Compare with your previous diagram. Which case (from posterior-predictive or from-posterior) has a wider mis-classification distribution? 
4. Is the classification at the true training data points (you can think of this as a bitstring '100101...' 90 characters long) represented in the posterior predictive trace? If so, how many times? Is it the most frequent string in the traces? Explain your conclusions.
5. Repeat 2 and 3 for the test set, i.e. make predictions. Describe and interpret the widths of the distributions?
 



## Extra Credit: Deriving the MLDT

If you took a distribution of prediction to your boss, he/she would probably ask you to make a choice. Decision Theory is the right way to make such a choice, rather than choosing an arbitrary set of bernoulli samplings. The idea is to minimize the posterior predictive averaged decision risk at each sample $x$:

$$R(g, x) = \int dy P(y \vert x) R(g, y \vert x) = \sum_{y=0,1} P(y \vert x) R(g, y \vert x)$$

where $g$ is the action you take: classify as 1 or 0. Use $R(g, y \vert x) = {\mathbb 1}_{g \ne y} \forall x$, the 1-0 loss.

1. Show that if $R(1, x) <= R(0, x)$ and thus we choose 1 as our classification, $p(1 \vert x) >= 0.5$. 
2. Use this MLDT rule and probabilities from both $p_{CDF}$ and $p_{PP}$ to make two classifications on the training set and test set. What is the misclassification rate in both? WHy do you think this is the case? 